In [7]:
! pip install transformers
! pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/7f/52/5b3b8d72c7a2087fee6849ecd0bf17b584241d7bc204aebd62c926f68064/torch-2.2.2-cp39-none-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/99/ff/c87e0622b1dadea79d2fb0b25ade9ed98954c9033722eb707053d310d4f3/sympy-1.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for mpmath<1.4,>=1.1.0 from https://files.pythonhosted.org/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae6

In [23]:
from transformers import BertModel, AutoTokenizer
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
model_name = "bert-base-cased"

In [3]:
model = BertModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/steve/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
text = "Tokenize me this please"

encoded_inputs = tokenizer(text=text, return_tensors="pt")
encoded_inputs

{'input_ids': tensor([[ 101, 1706, 6378, 3708, 1143, 1142, 4268,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
output = model(**encoded_inputs)


In [11]:
output.last_hidden_state

tensor([[[ 6.3545e-01,  1.7615e-01,  6.1589e-01,  ..., -1.1664e-01,
           5.5535e-01, -2.4577e-01],
         [ 2.6871e-01,  1.0804e-01, -7.9292e-02,  ...,  8.8401e-02,
           8.2027e-01, -7.8454e-04],
         [ 5.9982e-02,  5.7306e-01, -2.6445e-01,  ...,  2.0249e-01,
          -8.9708e-01,  2.3878e-01],
         ...,
         [-1.6705e-01,  3.0571e-01,  4.3537e-01,  ...,  8.0311e-02,
           2.3093e-01,  1.8002e-01],
         [ 4.5541e-01,  1.2220e-01,  5.8000e-01,  ...,  3.0526e-01,
           5.1684e-01, -2.6089e-01],
         [ 7.1147e-01,  3.9667e-02,  3.6133e-01,  ..., -4.8748e-01,
           5.2401e-01, -8.0548e-01]]], grad_fn=<NativeLayerNormBackward0>)

In [12]:
output.last_hidden_state.shape

torch.Size([1, 8, 768])

In [13]:
output.pooler_output.shape

torch.Size([1, 768])

In [51]:
def predict(input):
    encoded_inputs = tokenizer(text=input, return_tensors="pt")
    return model(**encoded_inputs).last_hidden_state

In [69]:
text1 = "There was a fly drinking from my soup"
text2 = "To become a commercial pilot, he had to fly for 1500 hours."

token1 = tokenizer.tokenize(text1)
token2 = tokenizer.tokenize(text2)

out1 = predict(text1)
out2 = predict(text2)

emb1 = out1[0, token1.index("fly"), :].detach()
emb2 = out2[0, token2.index("fly"), :].detach()

cosine(emb1, emb2) # a distance; min 0, max 1

0.4047783613204956

In [22]:
token1.index("fly")

3

In [81]:
text1 = "A steward is supposed to fly every day."
text2 = "To become a commercial pilot, he had to fly for 1500 hours."

token1 = tokenizer.tokenize(text1)
token2 = tokenizer.tokenize(text2)

out1 = predict(text1)
out2 = predict(text2)

emb1 = out1[0, token1.index("fly"), :].detach()
emb2 = out2[0, token2.index("fly"), :].detach()

cosine(emb1, emb2) # a distance; min 0, max 1

0.19182509183883667